## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [52]:
# Import all required libraries

import numpy as np
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier

In [53]:
train_datasets = np.array([49, 31, 67])
# test_datasets = np.array([55, 81, 47, 12, 4, 18, 28, 39, 66, 15, 78, 79, 30, 33, 11, 44])
test_datasets = np.array([55])

In [54]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


# Helper function to load dataset
def load_dataset(farm, event_id):
    X = np.loadtxt(f"./data/X_{farm}_{event_id}_10b.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data/y_{farm}_{event_id}_10b.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_dataset("C", dataset) for dataset in train_datasets])
    labels = np.concatenate([load_dataset_labels("C", dataset) for dataset in train_datasets])
    return dataset, labels


def get_test_dataset():
    dataset = load_dataset("C", 55)
    labels = load_dataset_labels("C", 55)
    return dataset, labels

In [55]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=1000,
    T=1000,
    s=10.0,
    max_included_literals=32,
    weighted_clauses=True,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    acc = tn / (fp + tn)

    print(f"Accuracy = {acc:.5f}")

    save_accuracy(epoch, acc, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {acc:.5f}")

    if acc > best_accuracy:
        best_accuracy = acc

        print("Saving model")
        save_model(tm, "best.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs


OverflowError: Python integer -1 out of bounds for uint32